
# Лекция 10
## Объектно-ориентированное программирование (продолжение)


__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2021 г.  

v 0.7 15.02.2022

## Разделы: <a class="anchor" id="разделы"></a>

* [Методы классов и статические переменные и методы](#методы-классов)    
* [Управление доступом к атрибутам класса](#управление-доступом)    
* [Динамические операции с атрибутами и интроспекция](#интроспекция)    
* [Специальные методы](#специальные)

In [1]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:lecture.css")
HTML(html.read().decode('utf-8'))


<p class="large_text" style="text-align:center"> Методы классов и статические переменные и методы </p>

<a class="anchor" id="методы-классов"></a>
* [к оглавлению](#разделы)

В прошлой лекции мы научились опеределять переменные (атрибуты) конкретного объекта класса. В этой лекции покажем, что мы можем создавать пременные, принадлежащие классу в целом.

Создадим класс корабля *Ship*, а в нем переменную класса *next_index*. Значение переменной *next_index* будет единым для всех кораблей, то есть для всех представителей класса *Ship*. Для этого используется метод *generate_next_index(cls)*, который увеличивает текущее значение *next_index* на единицу и возвращает значение *next_index* до увеличения. В методе *\_\_init\_\_(self)* каждому кораблю присваивается свой уникальный индекс на основе классовой переменной *next_index*.  Таким образом, у каждого корабля будет свой уникальный *index*, а у класса свой *next_index*.

В отличие от методов объекта, в которых первой переменной является *self*, в классовом методе первой переменной является *cls*.

Статический метод в Python (в отличие от других языков программирования) – это способ поместить некоторую функцию, которая выполняет что-то полезное для вашего класса, в пространство имен этого класса. В статический метод не передаются ни ссылка на объект *self*, ни ссылка на класс *cls*. В нашем примере статический метод проверяет разность индексов двух кораблей и сравнивает ее с 10. Если разность меньше, то корабли относятся к одной эпохе.

In [2]:
class Ship(object):
    next_index = 0  # переменная класса (статическая переменная)
    
    @classmethod
    def generate_next_index(cls): # в classmethod первый обязательный параметр: cls - переменная, ссылающаяся на КЛАСС
        index = cls.next_index
        cls.next_index += 1
        return index
    
    def __init__(self):
        self.index = Ship.generate_next_index()
        
    @staticmethod
    def is_from_same_epoch(sh1, sh2): # не имеет доступа ни к объекту ни к классу
        return abs(sh1.index - sh2.index) < 10

Создадим корабль и проверим его индекс.

In [3]:
s1 = Ship()
s1.index

0

При этом значение *next_index* класса увеличилось на 1 и стало равно 1.

In [4]:
Ship.next_index

1

Создадим флот из 15 кораблей.

In [5]:
fleet = [Ship() for _ in range(15)]

for sh in fleet:
    print(sh.index)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [6]:
Ship.next_index # доступ к переменной класса через имя класса

16

Для удобства *Python* позволяет к переменной класса обратиться и через объект.

In [7]:
s1.next_index # доступ к переменной класса через объект

16

Для всех экземпляров класса значение *next_index* одинаковое.

In [8]:
print([s.next_index for s in fleet])

[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]


Что если попытаться изменить значение *next_index*, используя объект? Переменная класса не изменится, зато будет создана новыя переменная *next_index* объекта.

In [9]:
fleet[0].next_index = 100 # приводит не к изменению в переменной класса, а к появлнию нового атрибута у данного объкта!
print([s.next_index for s in fleet], Ship.next_index)

[100, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16] 16


Это хорошо видно, если распечатать словарь атрибутов объекта. У объекта `fleet[0]` появился новый атрибут *next_index* со значением 100.

In [10]:
print(fleet[0].__dict__)
print(fleet[1].__dict__)

{'index': 1, 'next_index': 100}
{'index': 2}


Изменить переменную класса в явном виде все же можно, для этого надо использовать имя класса.

In [11]:
Ship.next_index = 50 # изменяем значение переменной класса
print([s.next_index for s in fleet], Ship.next_index)

[100, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50] 50


Также с помощью имени класса мы можем вызвать классовый метод *generate_next_index()*

In [12]:
Ship.generate_next_index() # доступ к методу класса через имя класса

50

Метод вернул текущее значение *next_index* и увеличил его на единицу.

In [13]:
Ship.next_index

51

Вызов классового метода возможен и с помощью имени любого объекта.

In [14]:
fleet[0].generate_next_index()

51

In [15]:
Ship.next_index

52

Посмотрим, как работает статический метод `is_from_same_epoch(sh1, sh2)`

In [16]:
print(fleet[0].index, fleet[-1].index)
Ship.is_from_same_epoch(fleet[0], fleet[-1])

1 15


False

In [17]:
print(s1.index, fleet[0].index)
s1.is_from_same_epoch(s1, fleet[0])

0 1


True

Следующий код демонстрирует то, что все корабли флота - это по сути разные объекты.

In [18]:
for s in fleet:
    print(s)

<p class="large_text" style="text-align:center"> Управление доступом к атрибутам класса </p>
    
<a class="anchor" id="управление-доступом"></a>
* [к оглавлению](#разделы)

В предыдущей лекции мы создали классы *Car* и *CargoCar*. Повторим этот код здесь.

In [19]:
class Car(object):
    """Базовый класс для автомобилей"""
    def __init__(self, x): # конструктор класса, используется для инициализации нового объекта
        self.x = x # создаем аттрибут класса        
        
    # метод класса; все методы класса должны в качестве первого атрибута иметь переменную self,
    # в которую автоматически передается ссылка на текущий объект 
    def is_near(self, x2): 
        return abs(self.x - x2) < 2.0 # self.x - обращение к атрибуту класса

# Класс, наследующий у Car
class CargoCar(Car): 
    def __init__(self, x, max_load, load):
        self.x = x
        self.max_load = max_load
        self.load = load
    
    def is_overloaded(self):
        return self.load > self.max_load

In [20]:
cc_1 = CargoCar(6.0, 10, 2)
cc_1.load, cc_1.max_load, cc_1.is_overloaded()

(2, 10, False)

In [21]:
# Изменяем загрузку грузового автомобиля и получаем сигнал о том, что автомобиль перегружен
cc_1.load = 12
cc_1.is_overloaded()

True

In [22]:
# Изменяем параметр максимальной загрузки, тем самым нарушаем безопасность перевозки
cc_1.max_load = 20
cc_1.is_overloaded()

False

Не следует давать обычным пользователям возможность изменять критически важные параметры, если это может привести к нарушению правил безопасности эксплуатации объекта.

В разных языках программирования этот вопрос решается по-разному. В *Python* управление доступом к методам осуществляется с помощью специальных имен самих методов. Примером является метод `__init__()`, имя которого начинается с двух подчеркиваний и заканчивается двумя подчеркиваниями.

Создадим перемнную, имя которой начинается с двух подчеркиваний. Доступ к такой переменной внутри класса осуществляется как обычно, а извне – ограничен. В следующем примере максимальная загрузка помещается в переменную `__max_load` при создании объекта, далее во время жизни объекта изменить это значение нельзя. 

In [23]:
# Класс CargoCar с контролем доступа к значениям загрузки и максимального предела загрузки 
class CargoCar2(Car): 
    def __init__(self, x, max_load, load):
        self.x = x
        self.__max_load = max_load
        self.__load = load
        assert not(self.is_overloaded()), 'При создании автомобиля превышено ограничение загрузки!'
    
    def is_overloaded(self):
        return self.__load > self.__max_load
    
    def get_load(self):
        return self.__load
    
    def set_load(self, load): # проверка при изменении значения
        assert load < self.__max_load, "Превышен предел загрузки!"
        self.__load = load
        
    def get_max_load(self): # для max_load есть только возможность получения значения
        return self.__max_load

Сделаем попытку создать грузовик с грузом, превышающим максимально допустимый.

In [24]:
сс2_1 = CargoCar2(5.0, 10, 11)

AssertionError: При создании автомобиля превышено ограничение загрузки!

Создадим грузовик, в котором нет превышения максимальной загрузки.

In [25]:
cc2_2 = CargoCar2(5.0, 10, 9)

Попытаемся получить значение загрузки, обратившись к именам объекта и его параметра.

In [26]:
cc2_2.__load # приватная переменная защищена от доступа извне класса

AttributeError: 'CargoCar2' object has no attribute '__load'

Не смотря на то, что такй параметр есть, мы получили сообщение, что его нет. Однако, мы создали метод *get_load()*, который возвращает значение этого параметра.

In [27]:
cc2_2.get_load()

9

Мы также создали метод *set_load()*, предназначенный для установки значения параметра `__load`.

In [28]:
cc2_2.set_load(8)
cc2_2.get_load()

8

Однако, установить загрузку выше максимальной не получится.

In [29]:
cc2_2.set_load(11)

AssertionError: Превышен предел загрузки!

В Python имеется более совершенный инструмент, позволяющий методы получения и установки (геттеры и сеттеры) скрытых параметров сделать более комфортным - это декораторы *@property* и *@имя_атрибута.setter*.

In [30]:
# Класс CargoCar с контролем доступа к значениям, выполненным в стиле Python
class CargoCar3(Car): 
    def __init__(self, x, max_load, load):
        self.x = x
        self.__max_load = max_load
        self.__load = load
        assert not(self.is_overloaded()), 'При создании автомобиля превышено ограничение загрузки!'
    
    def is_overloaded(self):
        return self.__load > self.__max_load
    
    @property # Декоратор функции, оформляющий функцию как функцию доступа
    def load(self):
        return self.__load
    
    @load.setter # Декоратор функции, оформляющий функцию как функцию-сеттер
    def load(self, val): # проверка при изменении значения
        assert val < self.__max_load, "Превышен предел загрузки!"
        self.__load = val
        
    # при необходимости, есть декоратор вида: @load.deletter
        
    @property 
    def max_load(self): # для max_load есть только возможность получения значения
        return self.__max_load

Создадим объект.

In [31]:
cc3_1 = CargoCar3(5.0, 10, 9)

Доступа к защищенной переменной нет.

In [32]:
cc3_1.__load

AttributeError: 'CargoCar3' object has no attribute '__load'

Однако, обращаясь как будто к атрибуту `load`, получаем значение атрибута `__load`. В этом случае срабатывает метод за декоратором *@property*.

In [33]:
cc3_1.load

9

Если же попытаться присвоить какое-либо значение переменной *load*, сработает метод *load(self, val)* под декоратором *@load.setter*.

In [34]:
cc3_1.load = 8
cc3_1.load

8

При попытке превысить максимально допустимую нагрузку автомобиля метод *load()* с декоратором *@load.setter* выдает ошибку.

In [35]:
cc3_1.load = 11

AssertionError: Превышен предел загрузки!

Таким образом, благодаря декораторам, работа пользователя с атрибутами класса выглядит естественно, хотя пользователь не знает, что на самом деле он работает с закрытыми атрибутами.

Получим значение переменной `__max_load`

In [36]:
cc3_1.max_load

10

Для `__max_load` мы сделали только геттер, но не сеттер. Поэтому получить значение `__max_load` мы смогли, но мы не можем установить новое значение.

In [37]:
cc3_1.max_load = 7

AttributeError: property 'max_load' of 'CargoCar3' object has no setter

## Динамические операции с атрибутами и интроспекция <a class="anchor" id="интроспекция"></a>
* [к оглавлению](#разделы)

Создадим несколько объектов и поместим их в список.

In [38]:
ob_1 = Car(3.1)
ob_2 = Car(4.1)
ob_3 = CargoCar3(5.0, 10, 9)
my_objects = [ob_1, ob_2, ob_3]

In [39]:
ob_1.x, ob_2.x, ob_3.x

(3.1, 4.1, 5.0)

Часто мы не знаем (в процессе выполнения программы), какие атрибуты есть у объекта. В предыдущей лекции у одного из потомков класса *Car* был атрибут *length*. Возможно, этот атрибут есть у объекта *ob.1*, проверим.

In [40]:
ob_1.length

AttributeError: 'Car' object has no attribute 'length'

Оказалось, что такого атрибута нет.

In [41]:
# присваиваем объекту значение для нового атрибута
ob_1.length = 11

In [42]:
ob_1.length

11

In [43]:
# у других объектов этого же типа данный атрибут отсутствует:
ob_2.length

AttributeError: 'Car' object has no attribute 'length'

In [44]:
# атрибут у объекта можно не только создать, но и удалить:
del ob_1.length

In [45]:
ob_1.length

AttributeError: 'Car' object has no attribute 'length'

Интроспекция позволяет узнать структуру объекта в процессе выполнения программы. Чтобы не получить исключение при обращении к несуществующему атрибуту можно заранее узнать о его наличии в объекте.

Получать, задвать и удалять значения атрибутов можно по их именам, хранящимся в виде строки, при помощи встроенных функций: 

- getattr()
- setattr()
- delattr()

Напишем код, в котором каждому объекту из списка *my_objects* присвоим атрибут *number*, содержащий порядковый номер согласно позиции в списке.

In [46]:
ob_1 = Car(3.1)
ob_2 = Car(4.1)
ob_3 = CargoCar3(5.0, 10, 9)
my_objects = [ob_1, ob_2, ob_3]

new_attr = 'number'
for i, o in enumerate(my_objects):
    setattr(o, new_attr, i)

In [47]:
ob_2.number

1

In [48]:
getattr(ob_2, new_attr)

1

In [49]:
getattr?

Данные функции позволяют обращаться к атрибутам, имена которых заранее неизвестны. Это особенно важно для реализации интроспекции. Интроспекция (type introspection) в программировании — возможность в объектно-ориентированных языках определить тип и структуру объекта во время выполнения программы. Эта возможность присуща языкам, позволяющих манипулировать типами объектов как объектами первого класса (first class citizens). 

Функция `getattr(object, name[, default])` выдает значение необязательного аргумента *default*, если атрибут отсутствует.

In [50]:
getattr(ob_2, 'length', 'Атрибута length нет')

'Атрибута length нет'

Если мы хоти узнать, какие вообще атрибуты и методы есть у объекта, можно воспользоваться встроенной функцией *dir()*.

In [51]:
dir(ob_3)

['_CargoCar3__load',
 '_CargoCar3__max_load',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'is_near',
 'is_overloaded',
 'load',
 'max_load',
 'number',
 'x']

In [52]:
?dir

In [55]:
# получаем значения и тип всех незащищенных переменных объекта:
for v_name in dir(ob_3):
    if v_name[0] == '_':
        continue
    attr = getattr(ob_3, v_name)
    print(v_name, attr, type(attr))

is_near <bound method Car.is_near of <__main__.CargoCar3 object at 0x000001B968CF94D0>> <class 'method'>
is_overloaded <bound method CargoCar3.is_overloaded of <__main__.CargoCar3 object at 0x000001B968CF94D0>> <class 'method'>
load 9 <class 'int'>
max_load 10 <class 'int'>
number 2 <class 'int'>
x 5.0 <class 'float'>


Такой результат плохо "читается". Попробуем повысить читаемость, для этого используем библиотеку *types* и отдельно обработаем мтоды и атрибуты.

In [56]:
# получаем значения и тип всех незащищенных переменных объекта:
import types

for v_name in dir(ob_3):
    if v_name[0] == '_':
        continue
    attr = getattr(ob_3, v_name)
    attr_t = type(attr)
    if attr_t is types.MethodType:
        print(v_name, '(method)', attr_t)
    else:
        print(v_name, attr, attr_t)

is_near (method) <class 'method'>
is_overloaded (method) <class 'method'>
load 9 <class 'int'>
max_load 10 <class 'int'>
number 2 <class 'int'>
x 5.0 <class 'float'>


Функция *vars()* возвращает атрибуты объекта для которых можно и получить и задать значения.

In [57]:
vars(ob_3)

{'x': 5.0, '_CargoCar3__max_load': 10, '_CargoCar3__load': 9, 'number': 2}

Наличие атрибута можно проверить с помощью функции *hasattr()*.

In [58]:
for o in my_objects:
    if hasattr(o, 'load'):
        print(o.number, o.load)

2 9


Встроенные функции для выполнения задач объектно-ориентированного программирования:

http://python-reference.readthedocs.io/en/latest/docs/functions/#object-oriented-functions


https://docs.python.org/3/library/functions.html


## Специальные методы <a class="anchor" id="специальные"></a>
* [к оглавлению](#разделы)

Методы `__repr__(self)` и `__str__(self)` служат для преобразования объекта в строку. Метод `__repr__()`  вызывается при выводе в интерактивной оболочке, когда распечатывается последняя строка в ячейке, а также при исnользовании функции *repr()*. Метод `__str__()` вызывается при выводе с помощью функции *print()*, а также при исnользовании функции *str()*. Если метод `__str__()` отсутствует, то будет вызван метод `__repr__()`. В качестве значения методы `__repr__()` и `__str__()` должны возвращать строку. Причем, значение возвращаемое `__repr__()` по возможности должно возврващать строку имеющую вид конструктора аналогичного объекта. То есть должно быть истинно выражание: `eval(repr(obj)) == obj`. 

Функция `eval()` в Python позволяет выполнить выражение, записанное в виде строки.

In [59]:
eval('[11, 22]+[33]')

[11, 22, 33]

In [60]:
for s in fleet:
    print(s)

Создадим класс *ShipS(Ship)* с методом `__str__()` и посмотрим, как изменится вывод с помощью функции *print()*. Класс *ShipS* является потомком класса *Ship*.

In [62]:
class ShipS(Ship):
    def __str__(self):
        return f'Ship with index {self.index}'

In [63]:
fleet2 = [ShipS() for _ in range(5)]
for s in fleet2:
    print(s)

Ship with index 52
Ship with index 53
Ship with index 54
Ship with index 55
Ship with index 56


Когда объект используется в строке формата, вызывается метод \__format\__(self, format_spec) объекта с самим объектом и  спецификацией формата в виде аргументов. Метод возвращает строку с экземпляром, отформатированным  соответствующим образом.

Специальные методы для поддержки преобразования типов:

* \_\_bооl_\_(self) – вызывается при исnользовании функции *bool()* 
* \_\_int_\_(self) – вызывается nри преобразовании объекта в целое число с nомощью функции *int()*
* \_\_float_\_(self) – вызывается nри nреобразовании объекта в вещественное число с nомощью функции *float()*
* \_\_complex_\_(self) – вызывается nри исnользовании функции *complex()*

Специальные методы для поддержки операций сравнения:

* х == у – равно – х.\_\_еq\_\_(у) 
* х != у – не равно – х.\_\_nе\_\_(у)
* х < у – меньше – х.\_\_lt\_\_(y) 
* х > y – бoльшe – x.\_\_gt\_\_(y)
* х <= у – меньше или равно – х.\_\_lе\_\_(у)
* х >= у – больше или равно – x.\_\_ge\_\_(y)
* у in х – проверка на вхождение – х.\_\_contains_\_(у)

Интерпретатор Python будет автоматически подставлять метод \_\_ne\_\_() (not equal – не равно), реализующий действие оператора неравенства (!=), если в классе присутствует реализация метода \_\_eq\_\_(), но отсутствует реализация метода \_\_ne_\_().

По умолчанию экземпляры наших собственных классов поддерживают оператор == (который всегда возвращает False) и являются хешируемыми (поэтому они могут использоваться в качестве ключей словаря или добавляться в множества). Но если реализовать специальный метод \_\_eq\_\_(), выполняющий корректную проверку на равенство, экземпляры перестанут быть хешируемыми. Это можно исправить, реализовав специальный метод \_\_hash\_\_(). Язык Python предоставляет функцию хеширования строк, чисел, фиксированных множеств и других классов. 

Специальный метод \_\_del\_\_(self) вызывается при уничтожении объекта - по крайней мере в теории. На практике метод \_\_del\_\_() может не вызываться никогда, даже при завершении программы. 

### Материалы для подготовки к следующей лекции:

Прохоренок: Глава 11 "Пользовательские функции"

Саммерфильд: Глава 8 "Улучшенные приемы программирования" (разделы: Улучшенные приемы процедурного программирования; Функциональное программирование